In [18]:
# 层和块
# 首先是多层感知机

import torch
from torch import nn
from torch.nn import functional as F #不包括层数的函数

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20) #input是一个2x20的随机矩阵
net(X)
# Sequential这个类是定义了一个特殊的Module
# 任何一个层或者神经网络都是Module的一个子类

# 自定义块
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)#调用父类, 不需要再初始化参数
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X))) # 将input依次传入相应的层
    
net = MLP()
net(X)

tensor([[-0.0741,  0.0915,  0.1115, -0.0320, -0.1141, -0.0944,  0.1504,  0.2984,
         -0.0931,  0.0342],
        [-0.1789,  0.0005,  0.1922,  0.0246, -0.0199,  0.0761,  0.1616,  0.2661,
         -0.1173, -0.0183]], grad_fn=<AddmmBackward0>)

In [12]:
# 顺序块, 调用Sequential这个类是如何实现的
class MySequential(nn.Module):
    def __init__(self, *args): # 这里*args是传入的layer
        super().__init__()
        for block in args: # 将每个layer依次加入列表list
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values(): # 这里因为是ordered，所以插入的list和返回的是一样的
            X = block(X) # 这里按列表list的顺序将每个layer放入
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[ 0.1682,  0.1343, -0.0377, -0.1447,  0.1132, -0.0507,  0.0475,  0.1033,
          0.0984, -0.0239],
        [ 0.1134,  0.0747,  0.0968,  0.0522,  0.1162, -0.0161,  0.0065,  0.1291,
         -0.0127,  0.0151]], grad_fn=<AddmmBackward0>)

In [13]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 随机生成weight, 但是不参与训练
        self.linear = nn.Linear(20, 20) 
        
    def forward(self, X):
        X = self.linear(X) # 将X放进来, 先做linear
        X = F.relu(torch.mm(X, self.rand_weight) + 1) # 这里手写X和random weight做乘法+1
        X = self.linear(X) # 做linear
        while X.abs().sum() > 1: #当绝对值>1, 一直变小
            X /= 2
        return X.sum() #返回一个标量

net = FixedHiddenMLP()
net(X)

tensor(0.0638, grad_fn=<SumBackward0>)

In [17]:
# 混合搭配各种组合块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3298, grad_fn=<SumBackward0>)

In [20]:
# 参数管理
# 首先关注具有单隐藏层的多层感知机
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

# 参数访问
print(net[2].state_dict()) # net[0]是Linear, net[1]是ReLU（但是ReLu没有参数), net[2]就是Linear

OrderedDict([('weight', tensor([[-0.2121,  0.1722,  0.1515, -0.1713, -0.2180, -0.2744, -0.2698, -0.2928]])), ('bias', tensor([0.0306]))])


In [23]:
# 目标参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)
print(net[2].weight.grad == None) #没有梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.0306], requires_grad=True)
tensor([0.0306])
True


In [28]:
# 一次性访问所有参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 这里的named_parameters()是能拿到一个字符串关于name, 有一个字符串关于param
# 这里*是解包, 将这个从dict变为items
net.state_dict()['2.bias'].data

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([0.0306])

In [30]:
# 从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), 
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.4156],
        [-0.4156]], grad_fn=<AddmmBackward0>)

In [31]:
# 我们已经设计了网络, 让我们看看是如何组合的
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [32]:
# 内置初始化 (修改一些内置，初始的函数)
def init_normal(m):
    if type(m) == nn.Linear: # m是一个module, 如果是一个全连接, 做normal dist(0, 0.01)
        nn.init.normal_(m.weight, mean=0, std=0.01) #这里'_'是一个替换函数, 不返回值
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0097,  0.0009,  0.0069, -0.0031]), tensor(0.))

In [33]:
def init_constant(m):
    if type(m) == nn.Linear: # m是一个module, 如果是一个全连接, 全都变成1
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
        
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

# 为什么weight不能全都一样, 因为会破坏对称性, 这样就相当于只有一个神经元

(tensor([1., 1., 1., 1.]), tensor(0.))

In [35]:
# 对某些块应用不同的初始化方法
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
        
net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.4103, -0.3058, -0.2120, -0.4948])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [38]:
# 自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print(
            "Init",
            *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.normal_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 # 保留绝对值大于4
    
net.apply(my_init)
net[0].weight[:2]

# 更简单的方法
net[0].weight.data[:] += 1 # 所有元素+1 
net[0].weight.data[0, 0] = 42 # 第一个元素变成42
net[0].weight.data[0]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([42.0000, -5.2473,  6.1025, -4.0458])

In [41]:
# 参数绑定
# 想在一些层中share parameters
# 先构造一个shared层
# 在构造中,可以理解为第2个隐藏层和第3个隐藏层是一个share层数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                   nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])
print(net[2] == net[4])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
True


In [42]:
# 自定义层
# 构建一个没有任何参数的自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()
    
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [43]:
# 将层作为组件并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-7.4506e-09, grad_fn=<MeanBackward0>)

In [44]:
# 带参数的图层
class MyLinear(nn.Module):
    def __init__(self, in_units, units): # 输入大小 # 输出大小
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 输出大小, 输出大小的初始化
        self.bias = nn.Parameter(torch.randn(units,)) # 输出大小的bias
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data # 这里通过.data访问对应的参数
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-1.8262, -1.0250,  0.2476],
        [-0.0225,  0.9352, -0.6843],
        [ 1.9856,  1.3576, -1.4438],
        [-1.1311, -0.5253, -0.6940],
        [ 0.5727,  0.4297,  0.2549]], requires_grad=True)

In [45]:
# 使用定义层直接执行正向传播计算
dense(torch.rand(2, 5))

tensor([[0.0000, 0.4704, 0.0000],
        [0.0000, 0.4045, 0.0000]])

In [47]:
# 使用自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[3.5218],
        [0.1191]])

In [52]:
# 读写和保存文件
x = torch.arange(4)
torch.save(x, 'x-file') # 在当前目录下存到x-file文件

x2 = torch.load('x-file') # 将这个file加载进来
x2

# 存储一个张量列表
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

# 写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [55]:
# 加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))
    
net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

# 将模型的参数存储为一个叫做"mlp.params"的文件
torch.save(net.state_dict(), 'mlp.params')

In [56]:
# 实例化了原始多层感知机的一个备份，直接读取文件中存储的参数

# 不仅仅要把存的参数带走, 还要把MLP的定义带走
clone = MLP() # 随机初始化了
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [57]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])